#### Objetivo
Neste caderno vamos utilizar a API do Compras Gvernamentais para identificar fracionamento de compras. O Fracionamento é uma tipologia de irregularidade da licitação que se caracteriza pela realização de compras idênticas no mesmo dia, pelo mesmo órgão, com soma dos valores superior ao limite de dispensa de licitação.

#### Pacotes utilizados

In [39]:
import pandas as pd ### Pacote para análise e transformação de tabelas de dados

### 1. Analisando tabela das licitações de jan a dez/2019 no Portal da Transparência
Fonte: http://portaldatransparencia.gov.br/download-de-dados/licitacoes

#### -> Carregando dados básicos das licitações

In [40]:
df_licitacoes = pd.read_csv('../_dados/compras/201901_Licitaá∆o.csv', sep = ';', encoding = 'latin1')

In [41]:
for i in ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
    df_temp = pd.read_csv('../_dados/compras/2019' + i + '_Licitaá∆o.csv', sep = ';', encoding = 'latin1')
    df_licitacoes = pd.concat([df_licitacoes, df_temp])

In [42]:
df_licitacoes.head(3)

,Número Licitação,Código UG,Código Modalidade Compra,Nome UG,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Município,Data Resultado Compra,Data Abertura,Valor Licitação
0,12017,160501,2,MUSEU HISTORICO DO EXERCITO FORTE COPACABANA,Tomada de Preços,64615003280201719,Objeto: Obra de restauração da rede de esgoto ...,Evento de Resultado de Julgame,52000,Ministério da Defesa,52121,Comando do Exército,RIO DE JANEIRO,17/01/2019,NaN,"564266,9500"
1,12017,764100,7,SERVICO DE IDENTIFICACAO DA MARINHA,Inexigibilidade de Licitação,63139000949201711,Objeto: Serviço de postagem de correspondências.,Encerrado,52000,Ministério da Defesa,52131,Comando da Marinha,RIO DE JANEIRO,23/01/2019,NaN,"5567,2000"
2,12018,927248,-99,FUNDO MUNICIPAL DE SAÚDE DE ANANINDEUA,Pregão - Registro de Preço,8915/2018,Objeto: Pregão Eletrônico - O objeto da prese...,Evento de Retificação Divulgad,99900,REPUBLICA FEDERATIVA DO BRASIL,93420,ESTADO DO PARA,Inválido,29/01/2019,30/11/2018,"71311,6900"


In [43]:
# Filtrando dispensas
df_licitacoes = df_licitacoes[df_licitacoes['Código Modalidade Compra'] == 6]

In [44]:
len(df_licitacoes)

111033

#### -> Carregando dados dos itens

In [45]:
df_itens = pd.read_csv('../_dados/compras/201901_ItemLicitaá∆o.csv', sep = ';', encoding = 'latin1')
for i in ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
    df_temp = pd.read_csv('../_dados/compras/2019' + i + '_ItemLicitaá∆o.csv', sep = ';', encoding = 'latin1')
    df_itens = pd.concat([df_itens, df_temp])

In [47]:
df_itens.head(3)

,Número Licitação,Código UG,Código Modalidade Compra,Nome UG,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,CNPJ Vencedor,Nome Vencedor,Descrição,Quantidade Item,Valor Item
0,12017,160501,2,MUSEU HISTORICO DO EXERCITO FORTE COPACABANA,Tomada de Preços,64615003280201719,52121,Comando do Exército,06241337000177,CRISART CONSTRUCOES E REFORMAS LTDA,MANUTENCAO DE REDES - AGUA / ESGOTO,1,"564266,9500"
1,12017,764100,7,SERVICO DE IDENTIFICACAO DA MARINHA,Inexigibilidade de Licitação,63139000949201711,52131,Comando da Marinha,34028316000294,EMPRESA BRASILEIRA DE CORREIOS E TELEGRAFOS,COMUNICACAO POR CORREIO,1,"5567,2000"
2,12018,927248,-99,FUNDO MUNICIPAL DE SAÚDE DE ANANINDEUA,Pregão - Registro de Preço,8915/2018,93420,ESTADO DO PARA,63848345000110,POLYMEDH. EIRELI,'PAPEL BOBINADO',1620,"8748,0000"


In [48]:
# Filtrando dispensas
df_itens = df_itens[df_itens['Código Modalidade Compra'] == 6]

In [49]:
len(df_itens)

338804

#### -> Criando tabela conjunta

In [51]:
# Criando id da compra nas tabelas de licitações e itens

def id_compra(x):
    y = str(x['Código UG']) + '06' + '0'*(9-len(str(x['Número Licitação']))) + str(x['Número Licitação'])
    return y

df_licitacoes['id_compra'] = df_licitacoes.apply(id_compra, axis = 1)
df_itens['id_compra'] = df_itens.apply(id_compra, axis = 1)

    
    
    

In [52]:
df_itens_compra = df_itens.merge(df_licitacoes[['id_compra', 'Data Resultado Compra']], how = 'inner', on = 'id_compra')


In [53]:
df_itens_compra.head(3)

,Número Licitação,Código UG,Código Modalidade Compra,Nome UG,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,CNPJ Vencedor,Nome Vencedor,Descrição,Quantidade Item,Valor Item,id_compra,Data Resultado Compra
0,12018,926397,6,AGÊN.BRAS.GEST.DE FUNDOS GARANTID.GARANT.S.A.,Dispensa de Licitação,004/2018,25000,MINISTERIO DA ECONOMIA,10242721000161,BUYSOFT DO BRASIL LTDA,LICENCIAMENTO DE DIREITOS PERMANENTES DE USO...,2,"6200,0000",92639706000012018,02/01/2019
1,12018,365001,6,FINANCIADORA DE ESTUDOS E PROJETOS,Dispensa de Licitação,012018,20502,Financiadora de Estudos e Projetos,04196935000812,GOLDEN DISTRIBUIDORA LTDA.,MONITOR IMAGEM,1,"2173,0800",36500106000012018,10/01/2019
2,12019,926690,6,CONSELHO REGIONAL DE ODONTOLOGIA - RJ,Dispensa de Licitação,01/2018,95320,ESTADO DO RIO DE JANEIRO,31302708000120,DIONE BEZERRA MENEZES - INFORMATICA,CARTUCHO TONER IMPRESSORA / COPIADORA XEROX,2,"300,0000",92669006000012019,09/01/2019


In [54]:
len(df_itens_compra)

338804

In [55]:
list(df_itens_compra)

['Número Licitação',
 'Código UG',
 'Código Modalidade Compra',
 'Nome UG',
 'Modalidade Compra',
 'Número Processo',
 'Código Órgão',
 'Nome Órgão',
 'CNPJ Vencedor',
 'Nome Vencedor',
 'Descrição',
 'Quantidade Item',
 'Valor Item',
 'id_compra',
 'Data Resultado Compra']

In [56]:
df_itens_compra['Valor Item'] = df_itens_compra['Valor Item'].apply(lambda x: float(x.replace(',', '.')))

In [57]:
df_itens_compra.to_pickle('df_itens_compra.pkl')

In [58]:
df_itens_compra = pd.read_pickle('df_itens_compra.pkl')

#### -> Identificando potenciais fracionamentos

In [60]:
df_itens_compra = df_itens_compra[df_itens_compra['Valor Item'] < 17600]


In [62]:
df_itens_compra_agregado = df_itens_compra[['id_compra', 'Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição', 'Valor Item']].groupby(['Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição']).agg({'Valor Item': 'sum', 'id_compra': 'count'}).reset_index()
df_itens_compra_agregado.columns = ['Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição', 'soma_valor', 'num_itens']


In [63]:
df_indicio_fracionamento = df_itens_compra_agregado[(df_itens_compra_agregado['soma_valor'] > 17600) & (df_itens_compra_agregado['num_itens'] > 1)]


In [64]:
df_indicio_fracionamento.head(3)

,Data Resultado Compra,Código UG,Nome UG,Descrição,soma_valor,num_itens
18,01/02/2019,114608,UNIDADE ESTADUAL DO IBGE NO MARANHãO,PRESTACAO DE SERVICO DE LIMPEZA E CONSERVACAO ...,41272.0,5
74,01/02/2019,155009,EBSERH HUB-UNB,MANUTENCAO GERADOR - VAPOR / GASOLINA,28100.0,6
83,01/02/2019,155021,EBSERH HU-UFMG,ANTICORPO,42133.0,4


In [65]:
len(df_indicio_fracionamento)

1587

Já conseguimos aqui chegar a 1587 indícios de fracionamento. Mas só com os dados do Portal não é possível se ter certeza, pois lá não é informado o inciso legal que justificou a dispensa. Lembrando que o fracionamento ocorre quando o motivo é o valor ser inferior ao limite que autoriza a compra direta, mas a ocorrência de outras compras idênticas fazem com que a soma dos valores supere esse limite.

Por isso, na próxima etapa vamos obter os incisos de justificativa lá na API do Compras Governamentais. Um questionamento válido seria "Por que não pegar logo essa informação para todas as compras, desde o incício?". O objetivo de chegarmos num grupo menor de indícios primeiro é economizar na utilização da API de compras, que não é muito rápida nas respostas das requisições.

### 2. Trazendo informação do inciso de justificativa da API do Compras Governamentais


#### -> Criando lista de id_compra de potenciais fracionamentos

In [75]:
df_itens_compra_ocorrencias = df_itens_compra.merge(df_indicio_fracionamento[['Data Resultado Compra', 'Código UG', 'Descrição']], how = 'inner', on = ['Data Resultado Compra', 'Código UG', 'Descrição'])


In [76]:
len(df_itens_compra_ocorrencias)

13805

In [77]:
lista_id_compra_ocorrencias = list(set(df_itens_compra_ocorrencias['id_compra']))

In [78]:
len(lista_id_compra_ocorrencias)

2405

#### -> Conectando API e trazendo informação do inciso
Não exige credencial. Só acessar o módulo desejado em: http://compras.dados.gov.br/docs/home.html

No nosso caso, queremos investigar as compras sem licitação: http://compras.dados.gov.br/docs/compraSemLicitacao/v1/compras_slicitacao.html

In [ ]:
raiz_url = 'http://compras.dados.gov.br/compraSemLicitacao/id/compra_slicitacao/'
id_compra = lista_id_compra_ocorrencias[0]
url = raiz_url + id_compra + '.csv'
df_compras_api = pd.read_csv(url)
df_compras_api['id_compra'] = id_compra

In [ ]:
count = 1

for i in lista_id_compra_ocorrencias[1:]:
    id_compra = i
    url = raiz_url + id_compra + '.csv'
    df_temp = pd.read_csv(url)
    df_temp['id_compra'] = id_compra
    df_compras_api = pd.concat([df_compras_api, df_temp])
    count+=1
    print(count, len(df_compras_api))
    

In [ ]:
len(df_compras_api)

In [ ]:
df_compras_api.to_pickle('df_compras_api.pkl')

In [ ]:
df_compras_api.head(3)

#### -> Agora, vamos juntar essa informação na nossa tabela de indícios

In [79]:
df_compras_api = pd.read_pickle('df_compras_api.pkl')

In [80]:
df_itens_compra_ocorrencias = df_itens_compra_ocorrencias.merge(df_compras_api[['id_compra', 'Número Inciso']], how = 'inner', on = 'id_compra')


### 3. Agora vamos refazer a lógica de identificação do fracionamento, mas somente para as potenciais ocorrências em que o inciso de justificativa foi o 2 (valor menor que limite).

In [81]:
df_itens_compra_ocorrencias = df_itens_compra_ocorrencias[df_itens_compra_ocorrencias['Número Inciso'] == 2]

In [82]:
len(df_itens_compra_ocorrencias)

2877

#### -> Agregando os dados para chegar aos fracionamentos

In [83]:
df_fracionamento = df_itens_compra_ocorrencias[['id_compra', 'Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição', 'Valor Item']].groupby(['Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição']).agg({'Valor Item': 'sum', 'id_compra': 'count'}).reset_index()
df_fracionamento.columns = ['Data Resultado Compra', 'Código UG', 'Nome UG', 'Descrição', 'soma_valor', 'num_itens']


In [84]:
df_fracionamento

,Data Resultado Compra,Código UG,Nome UG,Descrição,soma_valor,num_itens
0,01/02/2019,114608,UNIDADE ESTADUAL DO IBGE NO MARANHãO,PRESTACAO DE SERVICO DE LIMPEZA E CONSERVACAO ...,41272.00,5
1,01/02/2019,155009,EBSERH HUB-UNB,MANUTENCAO GERADOR - VAPOR / GASOLINA,28100.00,6
2,01/02/2019,155021,EBSERH HU-UFMG,ANTICORPO,42133.00,4
3,01/03/2019,158289,INST.FED.DO MARANHAO/CAMPUS ZE DOCA,SERVICO ENGENHARIA,11970.00,1
4,01/07/2019,240120,CENTRO BRASILEIRO DE PESQUISAS FISICAS,PRESTACAO DE SERVICOS DE PEDREIRO,29000.00,2
5,01/08/2019,753000,INSTITUTO DE EST. DO MAR ALTE PAULO MOREIRA,MICROCOMPUTADOR,22276.00,2
6,01/10/2019,254445,INSTITUTO DE TECNOLOGIA EM IMUNOBIOLOGICOS,EQUIPAMENTO / COMPONENTE - REFRIGERAÇÃO,21029.62,11
7,01/11/2019,153063,UNIVERSIDADE FEDERAL DO PARA,OBRAS CIVIS - PEQUENAS OBRAS / PINTURA EM GERAL,26700.00,2
8,01/11/2019,323028,AGENCIA NACIONAL DE ENERGIA ELETRICA,TREINAMENTO QUALIFICACAO PROFISSIONAL,20552.10,2
9,01/11/2019,785800,ESTACAO NAVAL DO RIO GRANDE,PEÇA/COMPONENTE MOTOR DE EMBARCAÇÃO,19065.30,76


In [85]:
df_fracionamento = df_fracionamento[(df_fracionamento['soma_valor'] > 17600) & (df_fracionamento['num_itens'] > 1)].reset_index(drop = 1)


In [86]:
df_fracionamento

,Data Resultado Compra,Código UG,Nome UG,Descrição,soma_valor,num_itens
0,01/02/2019,114608,UNIDADE ESTADUAL DO IBGE NO MARANHãO,PRESTACAO DE SERVICO DE LIMPEZA E CONSERVACAO ...,41272.00,5
1,01/02/2019,155009,EBSERH HUB-UNB,MANUTENCAO GERADOR - VAPOR / GASOLINA,28100.00,6
2,01/02/2019,155021,EBSERH HU-UFMG,ANTICORPO,42133.00,4
3,01/07/2019,240120,CENTRO BRASILEIRO DE PESQUISAS FISICAS,PRESTACAO DE SERVICOS DE PEDREIRO,29000.00,2
4,01/08/2019,753000,INSTITUTO DE EST. DO MAR ALTE PAULO MOREIRA,MICROCOMPUTADOR,22276.00,2
5,01/10/2019,254445,INSTITUTO DE TECNOLOGIA EM IMUNOBIOLOGICOS,EQUIPAMENTO / COMPONENTE - REFRIGERAÇÃO,21029.62,11
6,01/11/2019,153063,UNIVERSIDADE FEDERAL DO PARA,OBRAS CIVIS - PEQUENAS OBRAS / PINTURA EM GERAL,26700.00,2
7,01/11/2019,323028,AGENCIA NACIONAL DE ENERGIA ELETRICA,TREINAMENTO QUALIFICACAO PROFISSIONAL,20552.10,2
8,01/11/2019,785800,ESTACAO NAVAL DO RIO GRANDE,PEÇA/COMPONENTE MOTOR DE EMBARCAÇÃO,19065.30,76
9,02/04/2019,155021,EBSERH HU-UFMG,CURATIVO,22440.00,2


In [87]:
df_fracionamento.to_pickle('df_fracionamento.pkl')

É isso. Chegamos aqui a 556 grupos de aquisições (cada linha é 1 grupo) que compõem possíveis fracionamentos. São compras do mesmo material, no mesmo dia, pela mesma unidade gestora, justificadas (cada aquisição direta) pelo valor menor que R$ 17600, mas cuja soma dos valores supera esse limite. Então, o que se imagina é que o gestor tinha uma compra maior e fatiou-a para permitir a aquisição direta. Vale lembrar que são indícios. Para se confirmar é necessário uma análise mais pormenorizada dos processos de compra. Um possível fator que gera falso-positivo nesta análise é a descrição genérica do material comprado (deixei de propósito alguns exemplos acima, como "informação protegida por sigilo").

#### -> Visualizando detalhes das ocorrências

In [88]:
# Função para obter as informações detalhadas das compras de cada ocorrência

def detalha_ocorrencia(x):
    data = df_fracionamento.iloc[x]['Data Resultado Compra']
    ug = df_fracionamento.iloc[x]['Código UG']
    material = df_fracionamento.iloc[x]['Descrição']
    y = df_itens_compra_ocorrencias[(df_itens_compra_ocorrencias['Data Resultado Compra'] == data) &\
                                    (df_itens_compra_ocorrencias['Código UG'] == ug) &\
                                    (df_itens_compra_ocorrencias['Descrição'] == material)]
    return y

In [90]:
# Escolha um índice da tabela df_fracionamento para obter os detalhes:
detalha_ocorrencia(1)

,Número Licitação,Código UG,Código Modalidade Compra,Nome UG,Modalidade Compra,Número Processo,Código Órgão,Nome Órgão,CNPJ Vencedor,Nome Vencedor,Descrição,Quantidade Item,Valor Item,id_compra,Data Resultado Compra,Número Inciso
1051,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,9800.0,15500906111042019,01/02/2019,2
1052,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,2700.0,15500906111042019,01/02/2019,2
1053,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,2500.0,15500906111042019,01/02/2019,2
1054,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,3500.0,15500906111042019,01/02/2019,2
1055,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,4800.0,15500906111042019,01/02/2019,2
1056,111042019,155009,6,EBSERH HUB-UNB,Dispensa de Licitação,23522011104201893,26443,Empresa Brasileira de Serviços Hospitalares,72588080000159,POWER ENGENHARIA COMERCIO DE SERVICOS LTDA,MANUTENCAO GERADOR - VAPOR / GASOLINA,1,4800.0,15500906111042019,01/02/2019,2
